In [2]:
import os
import glob
import csv
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# From ultralytics import YOLO (for YOLO 11)
from ultralytics import YOLO

# TorchVision v2 transforms
from torchvision.transforms import v2
from torchvision.transforms.v2 import functional as F

In [ ]:
# Renaming Images to match the format FFFFF_IIIII.png

def rename_images(base_dir, start_folder_index=43):
    """
    Iterates through each folder under 'base_dir' and renames the contained images 
    to match the format FFFFF_IIIII.png, where:
      - FFFFF is a five-digit folder index (starting at 'start_folder_index')
      - IIIII is a five-digit image index (0-based) per folder
    """
    folder_paths = sorted(
        [f for f in glob.glob(os.path.join(base_dir, "*")) if os.path.isdir(f)]
    )

    current_folder_index = start_folder_index
    for folder_path in folder_paths:
        folder_name_formatted = f"{current_folder_index:05d}"
        
        image_paths = sorted(
            [img for img in glob.glob(os.path.join(folder_path, "*.png"))]
        )
        
        image_counter = 0
        for image_path in image_paths:
            image_name_formatted = f"{image_counter:05d}"
            new_name = f"{folder_name_formatted}_{image_name_formatted}.png"
            new_path = os.path.join(folder_path, new_name)
            os.rename(image_path, new_path)
            image_counter += 1
        
        current_folder_index += 1

In [ ]:
# Data Augmentation for Traffic Signs

class TrafficSignAugmentationPipeline:
    """
    Simple augmentation pipeline for traffic signs using torchvision.transforms.v2.
    Skips flips and large rotations since orientation is crucial for traffic signs.
    Emphasizes brightness, contrast, and minimal geometric changes.
    """
    def __init__(self):
        self.augment = v2.Compose([
            # Convert to Tensor first, so subsequent operations work on tensor data
            v2.ToImage(),
            
            # Random resizing within a range
            v2.RandomResize(
                min_size=int(0.5 * 256),
                max_size=int(1.0 * 256),
                interpolation=Image.BICUBIC
            ),
            
            # Adjust brightness/contrast to simulate varying lighting conditions
            v2.ColorJitter(
                brightness=0.4,
                contrast=0.4,
            ),
        
            
            # Convert to float and scale from [0, 255] to [0, 1]
            v2.ToDtype(torch.float32, scale=True),
        ])
        
    def __call__(self, img):
        """
        Applies all the transforms to the input image (tensor or PIL).
        Returns an augmented tensor image.
        """
        return self.augment(img)


def augment_images(base_dir, augmentation_pipeline):
    """
    Locates all .png images recursively in 'base_dir' and applies the provided 
    augmentation pipeline to each image. Saves augmented images in place by default.
    """
    image_paths = sorted(glob.glob(os.path.join(base_dir, "**", "*.png"), recursive=True))
    
    for img_path in image_paths:
        img = Image.open(img_path).convert("RGB")
        
        # Convert to tensor with v2.ToImage internally, or simply do the pipeline call
        img_tensor = F.to_tensor(img).float()
        aug_img = augmentation_pipeline(img_tensor)
        
        # Convert the augmented tensor back to PIL and overwrite the original
        aug_pil = F.to_pil_image(aug_img)
        aug_pil.save(img_path)

        # Alternatively, to save a separate augmented copy, uncomment:
        # new_path = img_path.replace(".png", "_aug.png")
        # aug_pil.save(new_path)

In [1]:
# YOLO 11 Detection and Visualization

def detect_and_visualize(base_dir, model_path="yolo11n.pt", conf_threshold=0.25, output_dir="output"):
    """
    Loads a YOLO 11 model (e.g., yolo11n.pt) via ultralytics and applies it 
    to each .png image under 'base_dir'.
    
    Returns detection results as (image_path, width, height, [boxes]) in a YOLO-like format:
      boxes -> [(class_id, x_center, y_center, box_width, box_height), ...]
    Visualizes and saves every 50th detection with bounding boxes in the output directory.
    """

    # Load YOLO 11 model from ultralytics
    model = YOLO(model_path)

    # The detection results are collected here
    detection_results = []

    all_image_paths = sorted(glob.glob(os.path.join(base_dir, "**", "*.png"), recursive=True))
    for idx, img_path in enumerate(all_image_paths):
        img = Image.open(img_path).convert("RGB")
        width, height = img.size

        # Perform inference using the model
        results = model.predict(source=img, conf=conf_threshold)

        # YOLO results come in a list, one per image. There's only one image here.
        boxes_info = []
        if len(results) > 0:
            r = results[0]
            # 'r.boxes' is a Boxes object containing bounding box coords, class, conf
            for box in r.boxes:
                # box.xyxy -> tensor of shape [1,4]
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                w_box = x2 - x1
                h_box = y2 - y1
                x_center = x1 + w_box / 2
                y_center = y1 + h_box / 2
                cls_id = int(box.cls.item())
                boxes_info.append((cls_id, x_center, y_center, w_box, h_box))

        detection_results.append((img_path, width, height, boxes_info))

        # Visualize and save every 50th image
        if idx % 50 == 0:
            fig, ax = plt.subplots(1)
            ax.imshow(img)
            for (cls_id, x_c, y_c, bw, bh) in boxes_info:
                rect = plt.Rectangle(
                    (x_c - bw/2, y_c - bh/2),
                    bw, bh,
                    fill=False,
                    color="red",
                    linewidth=2
                )
                ax.add_patch(rect)
            plt.title(f"Detections for {os.path.basename(img_path)}")

            # Save the figure
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            output_path = os.path.join(output_dir, f"detection_{idx}.png")
            plt.savefig(output_path)
            plt.close(fig)
        
    return detection_results

In [ ]:
# Converting YOLO-Format Boxes to dataset's CSV

def convert_detections_to_csv(detection_results, csv_filename="annotations.csv"):
    """
    Writes a CSV with columns:
      Width, Height, Roi.X1, Roi.Y1, Roi.X2, Roi.Y2, ClassId, Path
    Converting YOLO-like (x_center, y_center, w_box, h_box) to absolute 
    coordinates (x1, y1, x2, y2).
    """
    header = ["Width", "Height", "Roi.X1", "Roi.Y1", "Roi.X2", "Roi.Y2", "ClassId", "Path"]
    
    with open(csv_filename, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        
        for (img_path, img_w, img_h, boxes) in detection_results:
            for (class_id, x_center, y_center, bw, bh) in boxes:
                x1 = x_center - bw / 2
                y1 = y_center - bh / 2
                x2 = x_center + bw / 2
                y2 = y_center + bh / 2
                
                row = [
                    img_w,
                    img_h,
                    int(x1),
                    int(y1),
                    int(x2),
                    int(y2),
                    class_id,
                    img_path
                ]
                writer.writerow(row)

In [ ]:
########################
# 5) Main Execution Flow
########################
if __name__ == "__main__":
    base_dir = r"C:\Users\polat\Desktop\DATA" # Path to the dataset
    model_path = "traffic_signs.pt"   # YOLO 11 weight file
    
    # Step A: Rename images in existing folders (optional)
    # rename_images(base_dir, start_folder_index=43)
    
    # Step B: Augment images
    # augmentation_pipeline = TrafficSignAugmentationPipeline()
    # augment_images(base_dir, augmentation_pipeline)
    
    # Step C: Detection with YOLO 11
    detections = detect_and_visualize(base_dir, model_path=model_path, conf_threshold=0.25) # Saved in output folder
    
    # Step D: Conversion to CSV
    # convert_detections_to_csv(detections, csv_filename="annotations.csv")



0: 640x640 1 sign, 12.0ms
Speed: 3.1ms preprocess, 12.0ms inference, 62.6ms postprocess per image at shape (1, 3, 640, 640)

0: 608x640 1 sign, 66.8ms
Speed: 2.0ms preprocess, 66.8ms inference, 2.1ms postprocess per image at shape (1, 3, 608, 640)

0: 640x640 1 sign, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 544x640 1 sign, 56.4ms
Speed: 2.1ms preprocess, 56.4ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 640)

0: 576x640 1 sign, 58.5ms
Speed: 2.0ms preprocess, 58.5ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 sign, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)

0: 608x640 1 sign, 16.1ms
Speed: 3.0ms preprocess, 16.1ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 sign, 11.1ms
Speed: 3.0ms preprocess, 11.1ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608

In [11]:
# After replacing \ with /, and path with Test.csv alike path
# the following code is for extracting the class id from the path column
import pandas as pd
df = pd.read_csv("annotations.csv")
df["Path"]

# take the value between first and second / characters
df["Path"].str.split("/", expand=True)[1]

# put this values into the ClassId column
df["ClassId"] = df["Path"].str.split("/", expand=True)[1]

# save the updated dataframe to a new csv file
df.to_csv("annotations_updated.csv", index=False)